#요약 모델(pegasus)학습 및 추론 코드

In [1]:
# Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install datasets

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.8 MB/s eta 0:00:00


In [3]:
from transformers import PegasusTokenizer, PegasusForConditionalGeneration, Trainer, TrainingArguments, EarlyStoppingCallback
from datasets import load_dataset, Dataset, DatasetDict
from google.colab import drive
import torch
import numpy as np
import pickle
import pandas as pd
from torch.quantization import QuantStub, DeQuantStub
import os
import pickle

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

#데이터 병합 및 토크나이저, 처리(필요시 사용)


In [4]:
# # 병합

# # GPU가 사용 가능한지 확인
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(f"Using device: {device}")

# # 데이터 로드 및 병합
# data_folder_path = "/content/drive/MyDrive/summarizer/data"
# dataframes = []

# for filename in os.listdir(data_folder_path):
#     if filename.endswith(".csv"):
#         file_path = os.path.join(data_folder_path, filename)
#         df = pd.read_csv(file_path, encoding='latin1')

#         # 특정 열 이름만 바꾸기
#         if filename == "news_summary.csv":
#             df = df.rename(columns={
#                 "ctext": "article",  # 'ctext'를 'article'로 변경
#                 "text": "highlights"  # 'text'를 'highlights'로 변경
#             })

#         elif filename == "news_summary_more.csv":
#             df = df.rename(columns={
#                 "headlines": "highlights",  # 'headlines'를 'highlights'로 변경
#                 "text": "article"  # 'text'를 'article'로 변경
#             })

#         # 데이터프레임 리스트에 추가
#         dataframes.append(df)

# # 데이터프레임 병합
# combined_df = pd.concat(dataframes, ignore_index=True)

# # NaN을 빈 문자열로 대체
# combined_df['article'] = combined_df['article'].fillna("")

# # 'article' 열의 값이 문자열인지 확인하고, 문자열이 아닌 값은 빈 문자열로 대체
# combined_df['article'] = combined_df['article'].apply(lambda x: str(x) if isinstance(x, str) else "")

# # 빈 행(빈 문자열 포함)을 삭제
# combined_df = combined_df[combined_df['article'].str.strip() != ""]

# combined_df.to_csv("/content/drive/MyDrive/summarizer/data/combined_dataset.csv", index=False)


# dataset = Dataset.from_pandas(combined_df)

# train_test = dataset.train_test_split(test_size=0.1)
# datasets = DatasetDict({
#     'train': train_test['train'],
#     'test': train_test['test']
# })



In [5]:
# df = pd.read_csv('/content/drive/MyDrive/summarizer/data/combined_dataset.csv')

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(f"Using device: {device}")


# # Pegasus 모델 및 토크나이저 준비
# model_name = "google/pegasus-xsum"
# tokenizer = PegasusTokenizer.from_pretrained(model_name)
# model = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)

# # Pandas DataFrame을 Hugging Face Dataset으로 변환
# combined_dataset = Dataset.from_pandas(df)

# # 전처리 함수 정의 (GPU에서 입력 데이터를 처리하도록 변경)
# def preprocess_function(examples):
#     inputs = tokenizer(examples['article'], padding="max_length", truncation=True)
#     labels = tokenizer(examples['highlights'], padding="max_length", truncation=True)
#     inputs["labels"] = labels["input_ids"]
#     return inputs

# # 1. Hugging Face Datasets를 사용하여 데이터셋을 토크나이징 (배치 단위 처리)
# print("Tokenizing the dataset...")
# tokenized_datasets = combined_dataset.map(preprocess_function, batched=True)


In [6]:

# # 토크나이징된 데이터셋을 8개로 나눔
# dataset_length = len(tokenized_datasets)
# split_size = dataset_length // 8  # 데이터를 8등분

# # 3. 각 파트를 pkl 파일로 저장할 경로 설정
# output_dir = "/content/drive/MyDrive/summarizer/preprocessed/"
# os.makedirs(output_dir, exist_ok=True)

# for i in range(8):
#     start_idx = i * split_size
#     # 마지막 파트는 남은 모든 데이터를 포함
#     end_idx = (i + 1) * split_size if i < 7 else dataset_length  # 마지막 파트는 전체 끝까지 포함

#     # 각 파트의 데이터 추출 (Hugging Face Dataset에서 select 사용)
#     part_dataset = tokenized_datasets.select(range(start_idx, end_idx))

#     # pickle 파일로 저장
#     output_file = os.path.join(output_dir, f"processed_dataset_part_{i+1}.pkl")
#     with open(output_file, 'wb') as f:
#         pickle.dump(part_dataset, f)

#     print(f"Tokenized dataset part {i+1} saved to {output_file}")

#학습 세팅(토크나이징 된 데이터 받음)


In [7]:
# 모델과 데이터셋 설정
model_name = "google/pegasus-xsum"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/87.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.52M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/259 [00:00<?, ?B/s]

In [8]:
# 학습 파라미터 설정 (5 에폭씩 학습)
training_args = TrainingArguments(
        output_dir=f"/content/drive/MyDrive/summarizer/model",
        evaluation_strategy="epoch",  # 매 에포크마다 평가
        save_strategy="epoch",  # 매 에포크마다 체크포인트 저장
        learning_rate=2e-5,
        per_device_train_batch_size=4,  # 배치 크기 설정
        per_device_eval_batch_size=4,    # 배치 크기 설정
        num_train_epochs=5,  # 각각 5 에폭씩 학습
        weight_decay=0.01,
        logging_dir=f"/content/drive/MyDrive/summarizer/logging",
        logging_steps=10,
        save_total_limit=2,  # 최대 2개의 체크포인트만 저장
        load_best_model_at_end=True,  # 가장 성능이 좋은 모델 로드
        metric_for_best_model="eval_loss",  # 가장 좋은 모델을 선택할 기준
        greater_is_better=False  # 낮은 eval_loss가 더 좋은 모델임을 나타냄
    )

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [9]:
model_checkpoint_path = "/content/drive/MyDrive/summarizer/checkpoints"

In [10]:
last_checkpoint = None
if os.path.isdir(model_checkpoint_path) and os.listdir(model_checkpoint_path):
    last_checkpoint = Trainer.get_last_checkpoint(model_checkpoint_path)
    if last_checkpoint is not None:
        print(f"체크포인트에서 이어서 학습을 시작합니다: {last_checkpoint}")
    else:
        print("체크포인트가 없습니다. 새로 학습을 시작합니다.")
else:
    print("체크포인트 디렉토리가 존재하지 않습니다. 새로 학습을 시작합니다.")


체크포인트 디렉토리가 존재하지 않습니다. 새로 학습을 시작합니다.


#학습(학습 파일은 여러개로 되어있음)

In [11]:
# 데이터셋 리스트,
file_paths = [
    "/content/drive/MyDrive/combined/캡스톤 데이터 전처리 파일/summarizer/processed_dataset_part_1.pkl",
    "/content/drive/MyDrive/combined/캡스톤 데이터 전처리 파일/summarizer/processed_dataset_part_2.pkl",
    "/content/drive/MyDrive/combined/캡스톤 데이터 전처리 파일/summarizer/processed_dataset_part_3.pkl",
    "/content/drive/MyDrive/combined/캡스톤 데이터 전처리 파일/summarizer/processed_dataset_part_4.pkl",
    "/content/drive/MyDrive/combined/캡스톤 데이터 전처리 파일/summarizer/processed_dataset_part_5.pkl",
    "/content/drive/MyDrive/combined/캡스톤 데이터 전처리 파일/summarizer/processed_dataset_part_6.pkl",
    "/content/drive/MyDrive/combined/캡스톤 데이터 전처리 파일/summarizer/processed_dataset_part_7.pkl",
    "/content/drive/MyDrive/combined/캡스톤 데이터 전처리 파일/summarizer/processed_dataset_part_8.pkl"
]


In [12]:
def load_preprocessed_data(file_path):
    with open(file_path, 'rb') as f:
        tokenized_datasets = pickle.load(f)
    return tokenized_datasets

In [ ]:
#각 파일 가지고 학습시킴
for i, file_path in enumerate(file_paths):
    print(f"Loading dataset part {i+1} from {file_path}")


    loaded_data = load_preprocessed_data(file_path)



    # Trainer 설정
    trainer = Trainer(
        model=model,  # 일반 모델로 설정
        args=training_args,
        train_dataset=loaded_data,
        tokenizer=tokenizer,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]  # 조기 종료: 성능 개선 없으면 3번 후 종료
    )

    # 학습 시작
    print(f"Training on dataset part {i+1}...")
    try:
        trainer.train()
    except KeyboardInterrupt:
        print(f"Training interrupted during part {i+1}. Last checkpoint saved.")

    # api키는 이걸로 넣으면 됨 513a1f0c050fa7f60a76b5232e904d8df397082e

    # 체크포인트 저장
    trainer.save_model(f"/content/drive/MyDrive/summarizer/model_part_{i+1}")
    print(f"Model checkpoint saved after part {i+1}")


Loading dataset part 1 from /content/drive/MyDrive/combined/캡스톤 데이터 전처리 파일/summarizer/processed_dataset_part_1.pkl
Training on dataset part 1...


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss


In [ ]:
model.save_pretrained("/content/drive/MyDrive/summarizer/final_model")
tokenizer.save_pretrained("/content/drive/MyDrive/summarizer/final_model")
print("Final model and tokenizer saved.")

#입력 및 예측

In [ ]:
# 학습 시작 및 요약 결과 확인
def summarize_article(article_text):
    inputs = tokenizer(article_text, truncation=True, padding="longest", return_tensors="pt")
    summary_ids = model.generate(**inputs, max_length=60, num_beams=5, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary


In [ ]:
# 사용자 입력 문장을 요약
article_text = "여기다가 입력"
summary = summarize_article(article_text)
print("\nSummary:")
print(summary)